In [16]:
import openai
import pandas as pd
import json
from dotenv import load_dotenv
import os
import time

In [2]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# 현행 시설 및 프로그램 개선점 도출

In [3]:
df = pd.read_csv('Data/cleaned_data.csv')

In [4]:
# 개선점 도출 함수 정의
def extract_issues_from_review(review_text):
    prompt = f"다음 블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점을 찾아줘:\n\n{review_text}\n\n개선점:"
    
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    return response['choices'][0]['message']['content'].strip()


In [9]:
# 실험용 데이터
df.iloc[56]

location                                           천안시 동물보호센터
title                                 천안시, 동물보호센터 자원봉사자 상시 모집
url           https://blog.naver.com/fastcheonan/223545011360
postdate                                           2024-08-13
content     ​ 안녕하세요. 천안시입니다. ​ ​ 지난 3월유기·유실 동물의 입양과 건강한 반려...
Name: 56, dtype: object

In [10]:
# 실험용 코드1 - **돌리면 API 사용함**
review_text = df.iloc[56].content # 실험용 데이터
prompt = f"다음 블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점을 찾아줘:\n\n{review_text}\n\n개선점:"
    
response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=150,
    n=1,
    stop=None,
    temperature=0.5,
)

issues = extract_issues_from_review(review_text)

In [11]:
print(issues)

블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점은 명시되어 있지 않지만, 몇 가지 개선점을 제안할 수 있습니다:

1. **시설 유지 및 청결 관리**: 
   - 글에서 자원봉사자가 센터 환경정비에 참여한다고 언급되어 있습니다. 이는 시설의 청결과 유지 관리가 중요하다는 것을 의미합니다. 주기적인 청소와 정비 계획을 명확히 하고, 필요한 경우 추가적인 인력이나 자원을 투입할 필요가 있습니다.

2. **자원봉사자 교육 강화**:
   - 자원봉사자에게 기본 안전수칙과 유기견 돌봄 주의


>불만점 없어도 답변을 뱉음, 코드 수정

In [14]:
# 실험용 데이터
df.iloc[3]

location                                          도솔광장 반려인 쉼터
title                           천안 강아지 산책, 반려견 놀이터가 있는 천안축구센터
url               https://blog.naver.com/kimjymt/223310002749
postdate                                           2024-01-02
content     ​ ​ ​ ​ 에삐와 고미와 함께 천안에 있는 처가댁에 방문하여 며칠 신세를 졌다....
Name: 3, dtype: object

In [16]:
# 실험용 코드2 - **돌리면 API 사용함**
review_text = df.iloc[3].content # 실험용 데이터
prompt = f"다음 블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점을 찾아줘:\n\n{review_text}\n\n개선점:"
    
response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=150,
    n=1,
    stop=None,
    temperature=0.5,
)

issues = extract_issues_from_review(review_text)

In [18]:
print(issues)

블로그 글에서 언급된 천안축구센터의 개선점은 다음과 같습니다:

1. **진흙 문제**: 눈 비가 온 후 놀이터 입구 바닥이 질퍽하고 놀이터 안에도 진흙이 많아 날씨가 안 좋을 때는 놀기 어렵다는 점.
   - **개선 제안**: 놀이터 입구와 내부 바닥에 배수 시설을 개선하거나, 비가 온 후에도 사용할 수 있도록 인조 잔디나 배수 잘 되는 재질로 바닥을 보강하는 것이 좋겠습니다.

2. **쓰레기 관리**: 놀이터 안에 쓰레기통이 있음에도 불구하고 바


----

### 개선점 도출 함수 수정

In [13]:
# 함수 수정
# 개선점 도출 함수 정의
def extract_issues_from_review(review_text):
    prompt = f"""다음 블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점을 찾아줘:
    
    {review_text}

    만약 개선할 점이 명확하지 않거나 없으면 '개선점 없음'이라고 적고 끝내줘."""
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=240,  # 최대 토큰 수를 제한하여 짧게 응답하도록 설정
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    issues = response['choices'][0]['message']['content'].strip()
    
    if "개선점 없음" in issues or len(issues) < 10:  # 개선점이 없거나 짧은 경우
        return "개선점 없음"
    else:
        return issues

In [14]:
# 실험용
review_text = df.iloc[56].content
issues = extract_issues_from_review(review_text)
print(issues)

개선점 없음


In [17]:
# api 호출시 rate limit에 걸릴 경우 재시도하는 함수 정의
def extract_issues_with_retry(review_text, retries=3):
    for attempt in range(retries):
        try:
            return extract_issues_from_review(review_text)
        except openai.error.RateLimitError:
            print(f"Rate limit reached. Retrying in 20 seconds...")
            time.sleep(20)
    return "Rate limit exceeded, please try later."

In [18]:
# 전체 돌리기
# 장소별 개선점 도출 및 저장
location_improvement_points = {}

for location in df['location'].unique():
    location_reviews = df[df['location'] == location]
    improvement_points = []
    
    for idx, row in location_reviews.iterrows():
        url = row['url']
        review_text = row['content']
        issues = extract_issues_with_retry(review_text)
        
        improvement_points.append({
            'url' : url,
            'review': review_text,
            'issues': issues
        })
    
    if improvement_points:
        location_improvement_points[location] = improvement_points

Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 11012 tokens. Please reduce the length of the messages.

In [ ]:
# 일단 앞서 코드로 저장된 결과를 JSON 파일로 저장
with open('Data/location_improvement_points1.json', 'w', encoding='utf-8') as json_file:
    json.dump(location_improvement_points, json_file, ensure_ascii=False, indent=4)

print("Location improvement points have been saved to location_improvement_points.json")

In [37]:
len(improvement_points)

25

In [38]:
improvement_points[-1]

{'url': 'https://blog.naver.com/binggresmile/223246491651',
 'review': '\u200b 천안시는 오는 28일부터 29일까지 이틀간 오룡웰빙파크에서 천안시민과 함께하는 ‘K-DOG FESTIVAL’를 개최한다(포스터). \u200b 25일 시에 따르면, 이번 행사는 반려동물의 생명 존중 가치 확산과 성숙한 반려동물 문화 정착, 반려인과 비반려인·반려동물이 함께 즐길 수 있는 환경 조성을 위해 마련됐다. \u200b 천안시, 연암대학교, 백석문화대학교가 공동으로 주관하며 교육부가 지원하는 고등직업교육거점지구 사업(HiVE)의 일환으로 추진된다. \u200b 행사는 각종 경진대회, 반려동물 체험 부스, 전문가 특강 등 다양한 볼거리와 즐길 거리로 구성됐다. \u200b 천안시장배 학생 반려견 훈련 경진대회(고등부·대학부)를 시작으로  어질리티 경진대회(일반부)와 천안시민 누구나 참여할 수 있는 바른 산책대회, 반려견과 반려인이 함께하는 견공 올림픽 등이 개최될 예정이다. \u200b 올바른 반려문화 확산을 위해 ▲연암대 이웅종 교수와 함께하는 ‘바른 반려문화 산책 특강’ ▲펫 아로마 마사지 특강 ▲반려동물의 생각을 읽어보는 ‘펫타로’ ▲훈련·건강상담·미용 등 반려견과 함께 교감할 수 있는 다채로운 체험형 프로그램이 열린다. \u200b 성숙한 반려동물 문화정착을 위해 반려동물의 입양·등록 상담, 반려인의 에티켓 홍보 등도 진행한다. \u200b 또 반려동물 관련 업체와 천안시 동물호보센터, 충청남도 수의사회, 천안시 동물보호단체 등이 약 20여 개의 반려동물 산업관 부스를 운영해 다양한 볼거리를 제공할 예정이다. \u200b 시는 사람과 동물이 행복하게 살아가는 반려 문화 확산을 위해 축구센터와 백석동 일원에 반려인 쉼터를 조성하고 내년 1월 반려동물보호센터를 개장할 계획이다. \u200b 김영구 축산과장은 “이번 행사가 반려동물 인식 개선과 동물 보호, 생명 존중 가치 확산 등 성숙한 반려동물 문화 조성에 도움

In [34]:
df[df['url'] =='https://blog.naver.com/binggresmile/223246491651']

,location,title,url,postdate,content
52,백석동 반려인 쉼터,천안시 ‘K-DOG FESTIVAL’ 개최...“반려동물과 함께...,https://blog.naver.com/binggresmile/223246491651,2023-10-25,​ 천안시는 오는 28일부터 29일까지 이틀간 오룡웰빙파크에서 천안시민과 함께하는 ...


## 최종 개선점 도출 함수 수정

In [42]:
# 긴 리뷰 텍스트를 나누는 함수 정의
def split_text(text, max_tokens=3000):
    words = text.split()
    parts = []
    current_part = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # 단어와 공백의 길이를 고려
        if current_length > max_tokens:
            parts.append(' '.join(current_part))
            current_part = [word]
            current_length = len(word) + 1
        else:
            current_part.append(word)

    if current_part:  # 마지막 부분 추가
        parts.append(' '.join(current_part))

    return parts

In [43]:
# 개선점 도출 함수 정의
def extract_issues_from_review(review_text):
    parts = split_text(review_text)
    all_issues = []

    for part in parts:
        prompt = f"""다음 블로그 글에서 시설에 대한 불편했던 점이나 고쳐야 할 점을 찾아줘:

        {part}

        만약 개선할 점이 명확하지 않거나 없으면 '개선점 없음'이라고 적고 끝내줘."""
        
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=240,  # 최대 토큰 수를 제한하여 짧게 응답하도록 설정
            n=1,
            stop=None,
            temperature=0.5,
        )
        
        issues = response['choices'][0]['message']['content'].strip()
        
        if "개선점 없음" in issues or len(issues) < 10:  # 개선점이 없거나 짧은 경우
            continue  # '개선점 없음'은 추가하지 않음
        else:
            all_issues.append(issues)
    
    if not all_issues:
        return "개선점 없음" # 글 나눴을 때 각 부분에 대해서 전부 없을 경우
    else:
        return '\n'.join(all_issues)

# api 호출시 rate limit에 걸릴 경우 재시도하는 함수 정의
def extract_issues_with_retry(review_text, retries=3):
    for attempt in range(retries):
        try:
            return extract_issues_from_review(review_text)
        except openai.error.RateLimitError:
            print(f"Rate limit reached. Retrying in 20 seconds...")
            time.sleep(20)
        except openai.error.InvalidRequestError as e:
            print(f"Invalid request: {e}")
            return "Invalid request, please check the content length."
    return "Rate limit exceeded, please try later."

In [44]:
# 전체 돌리기 (df.iloc[53:]부터 시작)
# 장소별 개선점 도출 및 저장
location_improvement_points = {}

# 이미 처리한 부분을 건너뛰고, 53번째 행부터 시작
for idx, row in df.iloc[53:].iterrows():
    location = row['location']
    url = row['url']
    review_text = row['content']
    issues = extract_issues_with_retry(review_text)
    
    if location not in location_improvement_points:
        location_improvement_points[location] = []
        
    location_improvement_points[location].append({
        'url': url,
        'review': review_text,
        'issues': issues
    })

Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20

In [45]:
# 결과를 JSON 파일로 저장
with open('Data/location_improvement_points2.json', 'w', encoding='utf-8') as json_file:
    json.dump(location_improvement_points, json_file, ensure_ascii=False, indent=4)

print("Location improvement points have been saved to location_improvement_points.json")

Location improvement points have been saved to location_improvement_points.json


----

백석동이 뭔가 이상해서 다시..

In [49]:
location_improvement_points = {}

# 백석동만
for idx, row in df.iloc[53:].iterrows():
    location = row['location']
    if location == "백석동 반려인 쉼터":
        url = row['url']
        review_text = row['content']
        issues = extract_issues_with_retry(review_text)
        
        if location not in location_improvement_points:
            location_improvement_points[location] = []
            
        location_improvement_points[location].append({
            'url': url,
            'review': review_text,
            'issues': issues
        })

Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...
Rate limit reached. Retrying in 20 seconds...


In [ ]:
location_improvement_points

필요없을듯

# 데이터 합치기

In [52]:
# 두 개의 JSON 파일 로드
with open('Data/location_improvement_points1.json', 'r', encoding='utf-8') as f1:
    data_part1 = json.load(f1)

with open('Data/location_improvement_points2.json', 'r', encoding='utf-8') as f2:
    data_part2 = json.load(f2)

# JSON 데이터를 병합
merged_data = data_part1.copy()  # 첫 번째 파일의 내용을 복사

for location, improvement_points in data_part2.items():
    if location in merged_data:
        merged_data[location].extend(improvement_points)  # 중복된 키가 있으면 리스트에 추가
    else:
        merged_data[location] = improvement_points  # 중복된 키가 없으면 새로운 키로 추가

# 병합된 데이터를 새로운 JSON 파일로 저장
with open('Data/merged_location_improvements.json', 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=4)